In [1]:
import csv
import numpy as np
import pandas as pd

/var/folders/fb/xl0hnskx527bfqkv38l3sn7h0000gn/T/ipykernel_9295/3927477258.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def read_csv(file, read_empty_lines=True, formats={}, encoding='utf-8', delimiter=';', quotechar='"', verbose = True):
    with open (file=file, mode='r', encoding=encoding) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=delimiter, quotechar=quotechar)
        header = next(csv_reader)
        for i, column in enumerate(header):
            header[i] = column.strip().replace('\ufeff', '')
        n_columns = len(header)
        n_skipped_rows = 0
        skipped_row = []
        rows = []
        is_individual_format = False
        all_format = 0
        if 'all' in formats.keys():
            if formats['all'] == 'str':
                is_individual_format = False
                all_format = 1
        for i, row in enumerate(csv_reader):
            #print(', '.join(row))
            assert len(row) == n_columns, 'Počet sloupců je divný'
            for j, column in enumerate(header):
                if not is_individual_format and all_format == 0:
                    try:
                        cislo = int(row[j].strip().replace(' ', '').replace(',', '.'))
                    except Exception as e:
                        cislo = row[j].strip()
                        try:
                            cislo = float(row[j].strip().replace(' ', '').replace(',', '.'))
                        except Exception as e:
                            cislo = row[j].strip()
                    if str(cislo) != row[j].strip().replace(' ', '').replace(',', '.'):
                        cislo = row[j].strip()
                    #print(str(cislo), row[j].strip().replace(' ', '').replace(',', '.'))
                else:
                    cislo = row[j].strip()
                if j == 0:    #radky
                    if isinstance(cislo, str) and cislo == '' and not read_empty_lines:
                        skipped_row = ['']
                    else:
                        skipped_row = []
                        rows.append([cislo])
                else:    #cisla
                    if len(skipped_row) == 0:
                        rows[-1].append(cislo)
                    else:
                        if cislo == '':
                            skipped_row.append('')
                            if j == n_columns - 1: #last empty collumn in skipped row
                                n_skipped_rows += 1
                        else:
                            #not an empty row
                            rows.append(skipped_row)
                            skipped_row = []
                            rows[-1].append(cislo)
    assert csv_reader.line_num - 1 - n_skipped_rows == len(rows), 'Počet řádků je divný'
    if verbose:
        print(f'Skipped {n_skipped_rows} empty rows!')
    return rows, header

In [3]:
#file, delimiter = '_CR_242_PLS_M8_reader_data/CR_242_PLS_M8_old.csv', ';'
file, delimiter = '_NACE_data/nace.csv', '\t'

In [4]:
data, columns = read_csv(file, read_empty_lines=False, delimiter=delimiter)

Skipped 0 empty rows!


In [5]:
columns

['sekce', 'oddil', 'skupina', 'trida', 'nazev']

In [6]:
len(columns)

5

In [7]:
data

[['A', '', '', '', 'SEKCE A - ZEMĚDĚLSTVÍ, LESNICTVÍ A\xa0RYBÁŘSTVÍ'],
 ['A',
  '01',
  '',
  '',
  'Rostlinná a živočišná výroba, myslivost a související činnosti'],
 ['A', '01', '01.1', '', 'Pěstování plodin jiných než trvalých'],
 ['A',
  '01',
  '01.1',
  '01.11',
  'Pěstování obilovin (kromě rýže), luštěnin a olejnatých semen'],
 ['A', '01', '01.1', '01.12', 'Pěstování rýže'],
 ['A', '01', '01.1', '01.13', 'Pěstování zeleniny a melounů, kořenů a hlíz'],
 ['A', '01', '01.1', '01.14', 'Pěstování cukrové třtiny'],
 ['A', '01', '01.1', '01.15', 'Pěstování tabáku'],
 ['A', '01', '01.1', '01.16', 'Pěstování přadných rostlin'],
 ['A',
  '01',
  '01.1',
  '01.19',
  'Pěstování ostatních plodin jiných než trvalých'],
 ['A', '01', '01.2', '', 'Pěstování trvalých plodin'],
 ['A', '01', '01.2', '01.21', 'Pěstování vinných hroznů'],
 ['A', '01', '01.2', '01.22', 'Pěstování tropického a subtropického ovoce'],
 ['A', '01', '01.2', '01.23', 'Pěstování citrusových plodů'],
 ['A', '01', '01.2', '01

In [8]:
len(data)

1156

In [9]:
df = pd.DataFrame(data=data, columns=columns)

In [10]:
df

,sekce,oddil,skupina,trida,nazev
0,A,,,,"SEKCE A - ZEMĚDĚLSTVÍ, LESNICTVÍ A RYBÁŘSTVÍ"
1,A,01,,,"Rostlinná a živočišná výroba, myslivost a souv..."
2,A,01,01.1,,Pěstování plodin jiných než trvalých
3,A,01,01.1,01.11,"Pěstování obilovin (kromě rýže), luštěnin a ol..."
4,A,01,01.1,01.12,Pěstování rýže
...,...,...,...,...,...
1151,T,98,98.2,98.20,Činnosti domácností poskytujících blíže neurče...
1152,U,,,,SEKCE U - ČINNOSTI EXTERITORIÁLNÍCH ORGANIZACÍ...
1153,U,99,,,Činnosti exteritoriálních organizací a orgánů
1154,U,99,99.0,,Činnosti exteritoriálních organizací a orgánů


In [13]:
#df.drop(columns=[''], inplace=True)
df

,sekce,oddil,skupina,trida,nazev
0,A,,,,"SEKCE A - ZEMĚDĚLSTVÍ, LESNICTVÍ A RYBÁŘSTVÍ"
1,A,01,,,"Rostlinná a živočišná výroba, myslivost a souv..."
2,A,01,01.1,,Pěstování plodin jiných než trvalých
3,A,01,01.1,01.11,"Pěstování obilovin (kromě rýže), luštěnin a ol..."
4,A,01,01.1,01.12,Pěstování rýže
...,...,...,...,...,...
1151,T,98,98.2,98.20,Činnosti domácností poskytujících blíže neurče...
1152,U,,,,SEKCE U - ČINNOSTI EXTERITORIÁLNÍCH ORGANIZACÍ...
1153,U,99,,,Činnosti exteritoriálních organizací a orgánů
1154,U,99,99.0,,Činnosti exteritoriálních organizací a orgánů


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1156 entries, 0 to 1155
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   sekce    1156 non-null   object
 1   oddil    1156 non-null   object
 2   skupina  1156 non-null   object
 3   trida    1156 non-null   object
 4   nazev    1156 non-null   object
dtypes: object(5)
memory usage: 45.3+ KB


In [15]:
df[df['oddil'] == '']

,sekce,oddil,skupina,trida,nazev
0,A,,,,"SEKCE A - ZEMĚDĚLSTVÍ, LESNICTVÍ A RYBÁŘSTVÍ"
60,B,,,,SEKCE B - TĚŽBA A DOBÝVÁNÍ
103,C,,,,SEKCE C - ZPRACOVATELSKÝ PRŮMYSL
472,D,,,,"SEKCE D – VÝROBA A ROZVOD ELEKTŘINY, PLYNU, TE..."
492,E,,,,SEKCE E – ZÁSOBOVÁNÍ VODOU; ČINNOSTI SOUVISEJÍ...
512,F,,,,SEKCE F - STAVEBNICTVÍ
555,G,,,,SEKCE G - VELKOOBCHOD A MALOOBCHOD; OPRAVY A Ú...
689,H,,,,SEKCE H - DOPRAVA A SKLADOVÁNÍ
745,I,,,,"SEKCE I - UBYTOVÁNÍ, STRAVOVÁNÍ A POHOSTINSTVÍ"
772,J,,,,SEKCE J - INFORMAČNÍ A KOMUNIKAČNÍ ČINNOSTI


In [16]:
for i in df[df['oddil'] == ''].items():
    #print(type(i[0]), type(i[1]))
    #print(i[0], i[1].name)
    if i[0] == 'sekce':
        for j in i[1].items():
            print(j)
    if i[0] == 'nazev':
        for j in i[1].items():
            print(j)

(0, 'A')
(60, 'B')
(103, 'C')
(472, 'D')
(492, 'E')
(512, 'F')
(555, 'G')
(689, 'H')
(745, 'I')
(772, 'J')
(828, 'K')
(867, 'L')
(880, 'M')
(934, 'N')
(995, 'O')
(1012, 'P')
(1042, 'Q')
(1079, 'R')
(1106, 'S')
(1143, 'T')
(1152, 'U')
(0, 'SEKCE A - ZEMĚDĚLSTVÍ, LESNICTVÍ A\xa0RYBÁŘSTVÍ')
(60, 'SEKCE B - TĚŽBA A DOBÝVÁNÍ')
(103, 'SEKCE C - ZPRACOVATELSKÝ PRŮMYSL')
(472, 'SEKCE D – VÝROBA A ROZVOD ELEKTŘINY, PLYNU, TEPLA A\xa0KLIMATIZOVANÉHO VZDUCHU')
(492, 'SEKCE E – ZÁSOBOVÁNÍ VODOU; ČINNOSTI SOUVISEJÍCÍ S\xa0ODPADNÍMI VODAMI, ODPADY A SANACEMI')
(512, 'SEKCE F - STAVEBNICTVÍ')
(555, 'SEKCE G - VELKOOBCHOD A MALOOBCHOD; OPRAVY A\xa0ÚDRŽBA MOTOROVÝCH VOZIDEL')
(689, 'SEKCE H - DOPRAVA A SKLADOVÁNÍ')
(745, 'SEKCE I - UBYTOVÁNÍ, STRAVOVÁNÍ A\xa0POHOSTINSTVÍ')
(772, 'SEKCE J - INFORMAČNÍ A KOMUNIKAČNÍ ČINNOSTI')
(828, 'SEKCE K - PENĚŽNICTVÍ A POJIŠŤOVNICTVÍ')
(867, 'SEKCE L - ČINNOSTI V\xa0OBLASTI NEMOVITOSTÍ')
(880, 'SEKCE M - PROFESNÍ, VĚDECKÉ A\xa0TECHNICKÉ ČINNOSTI')
(934, 'SEKCE N - A

In [17]:
from collections import namedtuple
nace = namedtuple('nace', ['sekce', 'nazev'])
naces = []
for i in df[df['oddil'] == ''].index:
    #print(type(i[0]), type(i[1]))
    #print(i[0], i[1].name)
    naces.append(nace(df.iloc[i].sekce, df.iloc[i].nazev))

In [18]:
naces

[nace(sekce='A', nazev='SEKCE A - ZEMĚDĚLSTVÍ, LESNICTVÍ A\xa0RYBÁŘSTVÍ'),
 nace(sekce='B', nazev='SEKCE B - TĚŽBA A DOBÝVÁNÍ'),
 nace(sekce='C', nazev='SEKCE C - ZPRACOVATELSKÝ PRŮMYSL'),
 nace(sekce='D', nazev='SEKCE D – VÝROBA A ROZVOD ELEKTŘINY, PLYNU, TEPLA A\xa0KLIMATIZOVANÉHO VZDUCHU'),
 nace(sekce='E', nazev='SEKCE E – ZÁSOBOVÁNÍ VODOU; ČINNOSTI SOUVISEJÍCÍ S\xa0ODPADNÍMI VODAMI, ODPADY A SANACEMI'),
 nace(sekce='F', nazev='SEKCE F - STAVEBNICTVÍ'),
 nace(sekce='G', nazev='SEKCE G - VELKOOBCHOD A MALOOBCHOD; OPRAVY A\xa0ÚDRŽBA MOTOROVÝCH VOZIDEL'),
 nace(sekce='H', nazev='SEKCE H - DOPRAVA A SKLADOVÁNÍ'),
 nace(sekce='I', nazev='SEKCE I - UBYTOVÁNÍ, STRAVOVÁNÍ A\xa0POHOSTINSTVÍ'),
 nace(sekce='J', nazev='SEKCE J - INFORMAČNÍ A KOMUNIKAČNÍ ČINNOSTI'),
 nace(sekce='K', nazev='SEKCE K - PENĚŽNICTVÍ A POJIŠŤOVNICTVÍ'),
 nace(sekce='L', nazev='SEKCE L - ČINNOSTI V\xa0OBLASTI NEMOVITOSTÍ'),
 nace(sekce='M', nazev='SEKCE M - PROFESNÍ, VĚDECKÉ A\xa0TECHNICKÉ ČINNOSTI'),
 nace(sekce='N'

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1156 entries, 0 to 1155
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   sekce    1156 non-null   object
 1   oddil    1156 non-null   object
 2   skupina  1156 non-null   object
 3   trida    1156 non-null   object
 4   nazev    1156 non-null   object
dtypes: object(5)
memory usage: 45.3+ KB


In [20]:
#df['trida_nazev'] = df[['trida', 'nazev']].agg(' '.join, axis=1) #nejde!!!
df['skupina1'] = df['trida']
df.loc[df['skupina1'] == '', 'skupina1'] = df[df['skupina1'] == '']['skupina']
df['nazev1'] = df[['skupina1', 'nazev']].astype(str).agg(' '.join, axis=1)

In [21]:
df

,sekce,oddil,skupina,trida,nazev,skupina1,nazev1
0,A,,,,"SEKCE A - ZEMĚDĚLSTVÍ, LESNICTVÍ A RYBÁŘSTVÍ",,"SEKCE A - ZEMĚDĚLSTVÍ, LESNICTVÍ A RYBÁŘSTVÍ"
1,A,01,,,"Rostlinná a živočišná výroba, myslivost a souv...",,"Rostlinná a živočišná výroba, myslivost a sou..."
2,A,01,01.1,,Pěstování plodin jiných než trvalých,01.1,01.1 Pěstování plodin jiných než trvalých
3,A,01,01.1,01.11,"Pěstování obilovin (kromě rýže), luštěnin a ol...",01.11,"01.11 Pěstování obilovin (kromě rýže), luštěni..."
4,A,01,01.1,01.12,Pěstování rýže,01.12,01.12 Pěstování rýže
...,...,...,...,...,...,...,...
1151,T,98,98.2,98.20,Činnosti domácností poskytujících blíže neurče...,98.20,98.20 Činnosti domácností poskytujících blíže ...
1152,U,,,,SEKCE U - ČINNOSTI EXTERITORIÁLNÍCH ORGANIZACÍ...,,SEKCE U - ČINNOSTI EXTERITORIÁLNÍCH ORGANIZAC...
1153,U,99,,,Činnosti exteritoriálních organizací a orgánů,,Činnosti exteritoriálních organizací a orgánů
1154,U,99,99.0,,Činnosti exteritoriálních organizací a orgánů,99.0,99.0 Činnosti exteritoriálních organizací a or...


In [22]:
naces_dict = {}
for nace in naces:
    #print(nace.nazev)
    naces_dict[nace.nazev]=df[(df['sekce']==nace.sekce) & ((df['skupina'] != '') | (df['trida'] != ''))].nazev1.unique()

In [23]:
naces_dict

{'SEKCE A - ZEMĚDĚLSTVÍ, LESNICTVÍ A\xa0RYBÁŘSTVÍ': array(['01.1 Pěstování plodin jiných než trvalých',
        '01.11 Pěstování obilovin (kromě rýže), luštěnin a olejnatých semen',
        '01.12 Pěstování rýže',
        '01.13 Pěstování zeleniny a melounů, kořenů a hlíz',
        '01.14 Pěstování cukrové třtiny', '01.15 Pěstování tabáku',
        '01.16 Pěstování přadných rostlin',
        '01.19 Pěstování ostatních plodin jiných než trvalých',
        '01.2 Pěstování trvalých plodin', '01.21 Pěstování vinných hroznů',
        '01.22 Pěstování tropického a subtropického ovoce',
        '01.23 Pěstování citrusových plodů',
        '01.24 Pěstování jádrového a peckového ovoce',
        '01.25 Pěstování ostatního stromového a keřového ovoce a\xa0ořechů',
        '01.26 Pěstování olejnatých plodů',
        '01.27 Pěstování rostlin pro výrobu nápojů',
        '01.28 Pěstování koření, aromatických, léčivých a\xa0farmaceutických rostlin',
        '01.29 Pěstování ostatních trvalých plodin',

In [25]:
df[df['skupina'] == '01.4'].sort_values(by='trida')

,sekce,oddil,skupina,trida,nazev,skupina1,nazev1
22,A,01,01.4,,Živočišná výroba,01.4,01.4 Živočišná výroba
23,A,01,01.4,01.41,Chov mléčného skotu,01.41,01.41 Chov mléčného skotu
24,A,01,01.4,01.42,Chov jiného skotu,01.42,01.42 Chov jiného skotu
25,A,01,01.4,01.43,Chov koní a jiných koňovitých,01.43,01.43 Chov koní a jiných koňovitých
26,A,01,01.4,01.44,Chov velbloudů a velbloudovitých,01.44,01.44 Chov velbloudů a velbloudovitých
27,A,01,01.4,01.45,Chov ovcí a koz,01.45,01.45 Chov ovcí a koz
28,A,01,01.4,01.46,Chov prasat,01.46,01.46 Chov prasat
29,A,01,01.4,01.47,Chov drůbeže,01.47,01.47 Chov drůbeže
34,A,01,01.4,01.49,Chov ostatních zvířat,01.49,01.49 Chov ostatních zvířat
30,A,01,01.4,01.49.1,Chov drobných hospodářských zvířat,01.49.1,01.49.1 Chov drobných hospodářských zvířat


In [26]:
file, delimiter = '_ISCO_data/isco.csv', '\t'

In [27]:
data, columns = read_csv(file=file, delimiter=delimiter)

Skipped 0 empty rows!


In [28]:
columns

['hlavni_trida', 'trida', 'skupina', 'podskupina', 'kategorie', 'nazev']

In [29]:
data

[[0, '', '', '', '', 'Zaměstnanci v ozbrojených silách'],
 [0, '01', '', '', '', 'Generálové a důstojníci v ozbrojených silách'],
 [0, '01', '011', '', '', 'Generálové a důstojníci v ozbrojených silách'],
 [0, '01', '011', '0110', '', 'Generálové a důstojníci v ozbrojených silách'],
 [0, '01', '011', '0110', '01101', 'Generálové v ozbrojených silách'],
 [0, '01', '011', '0110', '01102', 'Vyšší důstojníci v ozbrojených silách'],
 [0, '01', '011', '0110', '01103', 'Nižší důstojníci v ozbrojených silách'],
 [0, '02', '', '', '', 'Poddůstojníci v ozbrojených silách'],
 [0, '02', '021', '', '', 'Poddůstojníci v ozbrojených silách'],
 [0, '02', '021', '0210', '', 'Poddůstojníci v ozbrojených silách'],
 [0, '02', '021', '0210', '02100', 'Poddůstojníci v ozbrojených silách'],
 [0,
  '03',
  '',
  '',
  '',
  'Zaměstnanci v ozbrojených silách (kromě generálů, důstojníků a poddůstojníků)'],
 [0,
  '03',
  '031',
  '',
  '',
  'Zaměstnanci v ozbrojených silách (kromě generálů, důstojníků a poddůs

In [30]:
df = pd.DataFrame(data=data, columns=columns)

In [31]:
df

,hlavni_trida,trida,skupina,podskupina,kategorie,nazev
0,0,,,,,Zaměstnanci v ozbrojených silách
1,0,01,,,,Generálové a důstojníci v ozbrojených silách
2,0,01,011,,,Generálové a důstojníci v ozbrojených silách
3,0,01,011,0110,,Generálové a důstojníci v ozbrojených silách
4,0,01,011,0110,01101,Generálové v ozbrojených silách
...,...,...,...,...,...,...
1985,9,96,962,9629,96291,Hlídači parkovišť a tržišť
1986,9,96,962,9629,96292,Uvaděči
1987,9,96,962,9629,96293,Šatnáři
1988,9,96,962,9629,96294,Toaletáři


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1990 entries, 0 to 1989
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   hlavni_trida  1990 non-null   int64 
 1   trida         1990 non-null   object
 2   skupina       1990 non-null   object
 3   podskupina    1990 non-null   object
 4   kategorie     1990 non-null   object
 5   nazev         1990 non-null   object
dtypes: int64(1), object(5)
memory usage: 93.4+ KB


In [33]:
df[df['trida'] == '']

,hlavni_trida,trida,skupina,podskupina,kategorie,nazev
0,0,,,,,Zaměstnanci v ozbrojených silách
18,1,,,,,Zákonodárci a řídící pracovníci
196,2,,,,,Specialisté
668,3,,,,,Techničtí a odborní pracovníci
1078,4,,,,,Úředníci
1187,5,,,,,Pracovníci ve službách a prodeji
1363,6,,,,,"Kvalifikovaní pracovníci v zemědělství, lesnic..."
1424,7,,,,,Řemeslníci a opraváři
1680,8,,,,,"Obsluha strojů a zařízení, montéři"
1877,9,,,,,Pomocní a nekvalifikovaní pracovníci


In [34]:
from collections import namedtuple
isco = namedtuple('isco', ['hlavni_trida', 'nazev'])
iscos = []
for i in df[df['trida'] == ''].index:
    #print(type(i[0]), type(i[1]))
    #print(i[0], i[1].name)
    iscos.append(isco(df.iloc[i].hlavni_trida, df.iloc[i].nazev))

In [35]:
iscos

[isco(hlavni_trida=0, nazev='Zaměstnanci v ozbrojených silách'),
 isco(hlavni_trida=1, nazev='Zákonodárci a řídící pracovníci'),
 isco(hlavni_trida=2, nazev='Specialisté'),
 isco(hlavni_trida=3, nazev='Techničtí a odborní pracovníci'),
 isco(hlavni_trida=4, nazev='Úředníci'),
 isco(hlavni_trida=5, nazev='Pracovníci ve službách a prodeji'),
 isco(hlavni_trida=6, nazev='Kvalifikovaní pracovníci v zemědělství, lesnictví a rybářství'),
 isco(hlavni_trida=7, nazev='Řemeslníci a opraváři'),
 isco(hlavni_trida=8, nazev='Obsluha strojů a zařízení, montéři'),
 isco(hlavni_trida=9, nazev='Pomocní a nekvalifikovaní pracovníci')]

In [36]:
df['podskupina1'] = df['kategorie']
df.loc[df['podskupina1'] == '', 'podskupina1'] = df[df['podskupina1'] == '']['podskupina']
df['nazev1'] = df[['podskupina1', 'nazev']].astype(str).agg(' '.join, axis=1)

In [37]:
df

,hlavni_trida,trida,skupina,podskupina,kategorie,nazev,podskupina1,nazev1
0,0,,,,,Zaměstnanci v ozbrojených silách,,Zaměstnanci v ozbrojených silách
1,0,01,,,,Generálové a důstojníci v ozbrojených silách,,Generálové a důstojníci v ozbrojených silách
2,0,01,011,,,Generálové a důstojníci v ozbrojených silách,,Generálové a důstojníci v ozbrojených silách
3,0,01,011,0110,,Generálové a důstojníci v ozbrojených silách,0110,0110 Generálové a důstojníci v ozbrojených silách
4,0,01,011,0110,01101,Generálové v ozbrojených silách,01101,01101 Generálové v ozbrojených silách
...,...,...,...,...,...,...,...,...
1985,9,96,962,9629,96291,Hlídači parkovišť a tržišť,96291,96291 Hlídači parkovišť a tržišť
1986,9,96,962,9629,96292,Uvaděči,96292,96292 Uvaděči
1987,9,96,962,9629,96293,Šatnáři,96293,96293 Šatnáři
1988,9,96,962,9629,96294,Toaletáři,96294,96294 Toaletáři


In [38]:
iscos_dict = {}
for isco in iscos:
    #print(nace.nazev)
    iscos_dict[isco.nazev]=df[(df['hlavni_trida']==isco.hlavni_trida) & ((df['podskupina'] != '') | (df['kategorie'] != ''))].nazev1.unique()

In [39]:
iscos_dict

{'Zaměstnanci v ozbrojených silách': array(['0110 Generálové a důstojníci v ozbrojených silách',
        '01101 Generálové v ozbrojených silách',
        '01102 Vyšší důstojníci v ozbrojených silách',
        '01103 Nižší důstojníci v ozbrojených silách',
        '0210 Poddůstojníci v ozbrojených silách',
        '02100 Poddůstojníci v ozbrojených silách',
        '0310 Zaměstnanci v ozbrojených silách (kromě generálů, důstojníků a poddůstojníků)',
        '03101 Praporčíci v ozbrojených silách',
        '03102 Mužstvo v ozbrojených silách',
        '03103 Čekatelé v ozbrojených silách',
        '03109 Ostatní zaměstnanci v ozbrojených silách (kromě generálů, důstojníků a poddůstojníků)'],
       dtype=object),
 'Zákonodárci a řídící pracovníci': array(['1111 Představitelé zákonodárné a výkonné moci',
        '11110 Představitelé zákonodárné a výkonné moci',
        '1112 Nejvyšší státní úředníci',
        '11121 Velvyslanci, konzulové a příbuzní pracovníci',
        '11122 Nejvyšší st

In [40]:
iscos_dict['Pomocní a nekvalifikovaní pracovníci']

array(['9111 Uklízeči a pomocníci v domácnostech (kromě hospodyní)',
       '91110 Uklízeči a pomocníci v domácnostech (kromě hospodyní)',
       '9112 Uklízeči a pomocníci v hotelích, administrativních, průmyslových a jiných objektech',
       '91121 Uklízeči a pomocníci v administrativních objektech',
       '91122 Uklízeči a pomocníci ve zdravotnických a sociálních zařízeních',
       '91123 Uklízeči a pomocníci v ubytovacích a vzdělávacích zařízeních',
       '91124 Uklízeči ve stravovacích zařízeních, potravinářských a farmaceutických výrobních prostorech',
       '91125 Uklízeči veřejných dopravních prostředků',
       '91126 Uklízeči výrobních prostor (kromě potravinářské a farmaceutické výroby) a skladů',
       '91127 Uklízeči prodejních prostor',
       '91128 Uklízeči v provozovnách osobních služeb',
       '91129 Ostatní uklízeči a pomocníci',
       '9121 Pracovníci pro ruční praní a žehlení',
       '91210 Pracovníci pro ruční praní a žehlení',
       '9122 Pracovníci pro